In [1]:
# Import modules
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import random
import skimage.transform
import os
import cv2
import os, cv2, re, random
import keras
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TRAIN_DIR = r"//192.168.5.8/Raman/UFC101/Images/train" 
TEST_DIR = r"//192.168.5.8/Raman/UFC101/Images/test" 
train_ucf = [TRAIN_DIR+'/'+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
test_ucf= [TEST_DIR+'/'+i for i in os.listdir(TRAIN_DIR)]

In [3]:
print('Total Training class: ',len(os.listdir(TRAIN_DIR)))
print('Total Testing class: ',len(os.listdir(TEST_DIR)))

Total Training class:  101
Total Testing class:  101


In [4]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

PICK First 10 classes

In [5]:
train_ucf.sort(key=natural_keys)
train_ucf = train_ucf[:10]
test_ucf.sort(key=natural_keys)

test_ucf = test_ucf[:10]

In [6]:
classes = [i+1 for i in range(10)]
classes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [7]:
train_list = [[0 for i in range(60)] for j in range(10)] 
train_list_y = [[j for i in range(60)] for j in range(10)] 

for j in range (10):
    
    for i in range(60):
        sting = train_ucf[j] + '/'
        train_list[j][i] = train_ucf[j] + '/'+os.listdir(train_ucf[j])[i]

In [8]:
train_list_y = np.array(train_list_y)

In [9]:
train_list_y.shape

(10, 60)

In [10]:
 def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)



In [11]:
shuffle_in_unison(train_list,train_list_y)

In [12]:
train_list[0][3], train_list_y[0][3]

('//192.168.5.8/Raman/UFC101/Images/train/8/v_Basketball_g08_c04.avi', 7)

In [13]:
test_list = [[0 for i in range(10)] for j in range(10)] 
for j in range (10): 
    for i in range(10):
        sting = test_ucf[j]
        test_list[j][i] = test_ucf[j]+ '/' +os.listdir(test_ucf[j])[i]

In [14]:
train_list_np = np.array(train_list)
test_list_np = np.array(test_list)
train_list_np.shape,  test_list_np.shape

((10, 60), (10, 10))

Every 10 classes has 60  training videos  and 10 testing videos

In [15]:
train_list_np_images = [[[0 for k in range(15)] for i in range(60)] for j in range(10)] 


In [16]:
for i in range(10):
    for j in range(60):
        for k in range(15):
            string = train_list_np[i][j] + '/' + os.listdir(train_list_np[i][j])[k+25]
            train_list_np_images[i][j][k]=(string)

In [17]:
train_list_np_images_np = np.array(train_list_np_images)

In [18]:
test_list_np_images = [[[0 for k in range(15)] for i in range(10)] for j in range(10)] 

for i in range(10):
    for j in range(10):
        for k in range(15):
            string = test_list_np[i][j] + '/' + os.listdir(test_list_np[i][j])[k+20]
            test_list_np_images[i][j][k]=(string)

In [19]:
test_list_np_images_np = np.array(test_list_np_images)
train_list_np_images_np.shape,test_list_np_images_np.shape

((10, 60, 15), (10, 10, 15))

In [20]:
train_list_np_images_np[6][59][5], train_list_y[6][59]

('//192.168.5.8/Raman/UFC101/Images/train/5/v_BalanceBeam_g21_c05.avi/Image_000030.jpg',
 4)

Open images

In [21]:
img_width = 299
img_height = 299
def prepare_data(list_of_images,class_num):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in (list_of_images):
        x.append(cv2.resize(cv2.imread(image), (img_size,img_size), interpolation=cv2.INTER_CUBIC))
        y.append(class_num)

 
    return np.array(x),np.array(y)

    Experiment with inception time distributed

In [22]:
# Parameters 
img_size =299
img_flat_size = img_size * img_size

# If you want to train the model -> True, otherwise -> False
Is_train = True

# If you want to load saved model -> True, otherwise -> False 
Load_model = False

# Name of the save file
save_name = 'soft1'

# Numbers of sampling to test the code 
num_test_sample = 10

# labels: 0 - 9
num_label = 10

# Parameters for training
num_epoch = 10

learning_rate = 5e-4
epsilon = 1e-8

batch_size = 256

# Parameter for LSTM
lstm_size = 256
step_size = 15
flatten_size = img_size

gpu_fraction = 0.3

In [23]:
# Initialize weights and bias 
def conv2d(x,w, stride):
	return tf.nn.conv2d(x,w,strides=[1, stride, stride, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# Get Variables
def weight_variable(name, shape):
    return tf.get_variable(name,shape=shape, initializer=tf.contrib.layers.xavier_initializer())

def bias_variable(name, shape):
    return tf.get_variable(name,shape=shape, initializer=tf.contrib.layers.xavier_initializer())

w = []

for i in range(15):
    weight = weight_variable('W'+str(i), [2048, 1])
    w.append(weight)

w

In [24]:
#LSTM Variables
time_stamps = 15
Wf=[]
Wi=[]
Wc = []
Wo = []

bf=[]
bf=[] 
bi =[]
bc=[]
bo=[]
Wa=[]
Wh=[]
for i in range(1,time_stamps+1):
    Wf.append(weight_variable('Wf'+str(i), [2048 + lstm_size, lstm_size])   ) # 2048 because Inception model's last layer has size : (?,8.8,2048)
    Wi.append(weight_variable('Wi'+str(i), [2048 + lstm_size, lstm_size]))
    Wc.append( weight_variable('Wc'+str(i), [2048 + lstm_size, lstm_size]))
    Wo.append(weight_variable('Wo'+str(i), [2048 + lstm_size, lstm_size]))

    bf.append( bias_variable('bf'+str(i), [lstm_size]))
    bi.append( bias_variable('bi'+str(i), [lstm_size]))
    bc.append (bias_variable('bc'+str(i), [lstm_size]))
    bo.append (bias_variable('bo'+str(i), [lstm_size]) )

    # Attention Variables
    Wa.append (weight_variable('Wa'+str(i), [2048, 1]))
    Wh.append( weight_variable('Wh'+str(i), [lstm_size, 1]))

In [25]:
Wa[5]

<tf.Variable 'Wa6:0' shape=(2048, 1) dtype=float32_ref>

In [26]:
# Soft Attention function
def soft_attention(h_prev, a, Wa, Wh):
    # h_prev: output from lstm of previous time step (shape: [batch_size, lstm_size])
    # a: Result of CNN [batch_size, conv_size * conv_size, channel_size] 
    
    m_list = [tf.tanh(tf.matmul(a[i], Wa) + tf.matmul(h_prev, Wh)) for i in range(len(a))] 
    m_concat = tf.concat([m_list[i] for i in range(len(a))], axis = 1)          
    alpha = tf.nn.softmax(m_concat) 
    z_list = [tf.multiply(a[i], tf.slice(alpha, (0, i), (-1, 1))) for i in range(len(a))]
    z_stack = tf.stack(z_list, axis = 2)                                                  
    z = tf.reduce_sum(z_stack, axis = 2)                                                  
    return alpha, z
    
    
    
    
# LSTM function
def LSTM_cell(C_prev, h_prev, x_lstm, Wf, Wi, Wc, Wo, bf, bi, bc, bo):
    
    
     #  rnn_state = C_prev
    
    
    # C_prev: Cell state from lstm of previous time step (shape: [batch_size, lstm_size])   
    # h_prev: output from lstm of previous time step (shape: [batch_size, lstm_size])
    # x_lstm: input of lstm (shape: [batch_size, data_flatten_size])

    input_concat = tf.concat([x_lstm, h_prev], 1)           
    f = tf.sigmoid(tf.matmul(input_concat, Wf) + bf)       
    i = tf.sigmoid(tf.matmul(input_concat, Wi) + bi)       
    c = tf.tanh(tf.matmul(input_concat, Wc) + bc)          
    o = tf.sigmoid(tf.matmul(input_concat, Wo) + bo)       
    
    
    C_t = tf.multiply(f, C_prev) + tf.multiply(i, c)       
    h_t = tf.multiply(o, tf.tanh(C_t))                      
    
    return C_t, h_t # Cell state, Output


In [27]:
x_image  = tf.placeholder(tf.float32, shape = [None,time_stamps, img_size,img_size, 3])
y_target = tf.placeholder(tf.float32, shape=[None, num_label])
x= keras.applications.inception_v3.preprocess_input(x_image)        #<tf.Tensor 'sub:0' shape=(?, 15, 299, 299, 3) dtype=float32>


In [28]:
with tf.variable_scope("InceptionModel"):
    base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)
inputs = tf.keras.Input(shape=(time_stamps,img_width,img_height,3) , tensor = x)
outputs = tf.keras.layers.TimeDistributed(base_model)(inputs)


In [29]:
last_layer = outputs
conv_size = last_layer.get_shape()[2] 
conv_flat = tf.reshape(last_layer, [-1,time_stamps, conv_size*conv_size, 2048])               
conv_unstack = tf.unstack(conv_flat, axis = 1)    

In [30]:
len(conv_unstack),conv_unstack[1]

(15, <tf.Tensor 'unstack:1' shape=(?, 64, 2048) dtype=float32>)

In [31]:
new_conv_unstack = tf.unstack(conv_unstack[0], axis = 1)

In [32]:
rnn_batch_size = tf.shape(x_image)[0]                                           

# Initial lstm cell state and output 

'''
rnn_state = []
rnn_out = []
for i in range(time_stamps):
    rnn_state.append (tf.zeros([ rnn_batch_size, lstm_size], tf.float32) )  
    rnn_out.append( tf.zeros([ rnn_batch_size, lstm_size], tf.float32)  )    
'''
rnn_state= (tf.zeros([ rnn_batch_size, lstm_size], tf.float32) )  
rnn_out=( tf.zeros([ rnn_batch_size, lstm_size], tf.float32)  )    
alpha = []
#################################### Attention!!! ####################################
for i in range(time_stamps):
    new_conv_unstack = tf.unstack(conv_unstack[i], axis = 1)    
    
    alpha_Individual, z = soft_attention(rnn_out, new_conv_unstack, Wa[i], Wh[i])     
    alpha.append(alpha_Individual)
    rnn_state, rnn_out = LSTM_cell(rnn_state, rnn_out, z, Wf[i], Wi[i], Wc[i], Wo[i], bf[i], bi[i], bc[i], bo[i])
######################################################################################


In [33]:
## Weights for output dense layer

w_fc1 = weight_variable('w_fc1',[lstm_size, num_label])
b_fc1 = bias_variable('b_fc1', [num_label])

In [34]:

output = tf.matmul(rnn_out, w_fc1)+b_fc1
output = tf.nn.softmax(output)
# Training 
Loss = tf.keras.backend.categorical_crossentropy(target = y_target, output = output)
Cost = tf.reduce_mean(Loss)
optimizer = tf.train.AdamOptimizer(learning_rate =learning_rate, epsilon = epsilon).minimize(Cost)

correct_prediction = tf.equal(tf.argmax(y_target,1), tf.argmax(output,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [35]:
# Create Session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpu_fraction
saver = tf.train.Saver()

sess = tf.keras.backend.get_session()
initialize_variables_list = list(set(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))^set(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='InceptionModel')))
writer = tf.summary.FileWriter("output", sess.graph)
sess.run(initialize_variables_list )

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],


In [36]:
# Training
#train_data_num =len(Train_list_all)   #(consider all training images)
#random_idx = np.arange(train_data_num)

if Is_train == True:

    for i in range(num_epoch):
        # Making batches
        
        batch_count = 1
        for class_num in range(10):
        
            for batch_num in range(0,60,4):
                x_train_batch = []
                y_train_batch = []

                for j in range(4):

                    train_batch_list = train_list_np_images_np[class_num][j+batch_num]
                    X_img, _ = prepare_data(train_batch_list,class_num)
                    x_train_batch.append(X_img)
                    y_train_batch.append(train_list_y[class_num][j+batch_num])
                    
                    
                train_batch = np.array(x_train_batch)    
                y_train_batch = np.array(y_train_batch)
                batch_y_train = tf.keras.utils.to_categorical(y_train_batch,num_classes=10)
            
            
            
            
                #rain_batch = np.reshape(batch_x_train, (batch_x_train.shape[0], img_size, img_size, 3))

                # Training
                optimizer.run(session=sess, feed_dict = {x_image: train_batch, y_target: batch_y_train})
                cost = Cost.eval( session=sess, feed_dict = {x_image: train_batch, y_target: batch_y_train})
                acc = accuracy.eval( session=sess, feed_dict = {x_image: train_batch, y_target: batch_y_train})

                print("Epoch: " + str(i+1) + ' / ' + 
                      "Batch: " + str(j)  + ' / ' + 
                      "Cost: " + str(cost) + ' / ' + 
                      "Training Accuracy: " + str(acc))
        print("End of epoch")



ResourceExhaustedError: OOM when allocating tensor with shape[60,35,35,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: time_distributed/batch_normalization_25/cond/FusedBatchNorm_1/Switch-0-1-TransposeNCHWToNHWC-LayoutOptimizer = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](time_distributed/batch_normalization_25/cond/FusedBatchNorm_1/Switch, PermConstNCHWToNHWC-LayoutOptimizer)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
train_batch.shape